In [42]:
import pandas as pd
import numpy as np
import joblib
import requests

from io import BytesIO
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import openpyxl

In [2]:
def imputar_f (df,list_cat,SimpleImputer,pd):  
    columns = df.columns.values
    df_c=df[list_cat]
    df_n=df.loc[:,~df.columns.isin(list_cat)]

    imputer_n=SimpleImputer(strategy='median')
    imputer_c=SimpleImputer(strategy='most_frequent')

    imputer_n.fit(df_n)
    imputer_c.fit(df_c)

    X_n=imputer_n.transform(df_n)
    X_c=imputer_c.transform(df_c)

    df_n=pd.DataFrame(X_n,columns=df_n.columns)
    df_c=pd.DataFrame(X_c,columns=df_c.columns)

    df = pd.concat([df_n,df_c],axis=1)[columns]
    return df

In [11]:
objects = ['RandomForestModel', 'Features', 'most_frequently', 'dummies']
for obj in objects:
  link_ = f'https://github.com/santiagogz11/HRProject/blob/main/{obj}.pkl?raw=true'
  file_ = BytesIO(requests.get(link_).content)
  globals()[obj] = joblib.load(file_)

In [12]:
DataNames = {'employee_survey_data':'employee_survey', 'general_data':'general', 'manager_survey_data':'manager_survey'}

In [13]:
for key, value in zip(DataNames.keys(), DataNames.values()):
  globals()[f'df_{value}'] = pd.read_csv(f'https://raw.githubusercontent.com/santiagogz11/HRProject/main/data/{key}.csv', engine='python', sep = None)

In [14]:
df = pd.merge(df_general, df_employee_survey, on = 'EmployeeID', how = 'left')
df = pd.merge(df, df_manager_survey, on = 'EmployeeID', how = 'left')

In [17]:
df = imputar_f(df, most_frequently, SimpleImputer, pd)

In [19]:
noDummies = df.columns[~df.columns.isin(most_frequently)].values

In [20]:
scaler = MinMaxScaler()
X_noDummies = scaler.fit_transform(df[noDummies])

In [21]:
X_dummies = pd.get_dummies(df[dummies])

In [23]:
X = pd.concat([pd.DataFrame(X_dummies), pd.DataFrame(X_noDummies, columns=noDummies)], axis = 1)[Features]

In [24]:
predict = pd.concat([pd.DataFrame(X.index.values + 1, columns=['EmployeeID']), 
                     pd.DataFrame(RandomForestModel.predict(X), columns=['Attrition'])], axis = 1)

In [25]:
predict.query('Attrition == 1')

,EmployeeID,Attrition
1,2,1
6,7,1
13,14,1
28,29,1
30,31,1
...,...,...
4381,4382,1
4386,4387,1
4388,4389,1
4391,4392,1


In [64]:
#
predict_f = pd.concat([predict, pd.DataFrame(RandomForestModel.predict_proba(X)).
           rename(columns={1:'Probability'})[['Probability']]], axis=1).query('Attrition == 1').sort_values(by='Probability', ascending=False)

In [66]:
predict_f

,EmployeeID,Attrition,Probability
1013,1014,1,1.00
1132,1133,1,1.00
4072,4073,1,1.00
2602,2603,1,1.00
2853,2854,1,1.00
...,...,...,...
739,740,1,0.60
30,31,1,0.60
2705,2706,1,0.52
4175,4176,1,0.52


In [65]:
#Se exporta el despliegue con la predicción de los empleados propensos a retirarse
from google.colab import drive
drive.mount('/content/drive')
predict_f.to_excel('/content/drive/MyDrive/AplicacionesAnalitica/HR/GitHub/HRProject/Prediction.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
#Se exporta el despliegue con la predicción de lla importancia de cada variable en el modelo
(pd.DataFrame(RandomForestModel.feature_importances_, 
             index=RandomForestModel.feature_names_in_, 
             columns=['importances']
    ).sort_values(by='importances', ascending=False)).to_excel('/content/drive/MyDrive/AplicacionesAnalitica/HR/GitHub/HRProject/Importance.xlsx')